In [1]:
import requests
import json
import pandas as pd
import numpy as np
import info
import statistics as stat
import time

In [2]:
access_token = info.access_token
account_id = info.account_id

static_api = info.static_api
stream_api = info.stream_api

header = {
    'Authorization': f'Bearer {access_token}'
}

In [3]:
session = requests.Session()

In [4]:
ins_df = pd.read_pickle('instruments.pkl')

In [5]:
ins_df

,name,type,displayName,pipLocation,marginRate
0,EUR_HUF,CURRENCY,EUR/HUF,-2,0.05
1,EUR_DKK,CURRENCY,EUR/DKK,-4,0.1
2,USD_MXN,CURRENCY,USD/MXN,-4,0.10
3,GBP_USD,CURRENCY,GBP/USD,-4,0.03
4,CAD_CHF,CURRENCY,CAD/CHF,-4,0.04
...,...,...,...,...,...
63,GBP_AUD,CURRENCY,GBP/AUD,-4,0.03
64,USD_PLN,CURRENCY,USD/PLN,-4,0.05
65,CAD_HKD,CURRENCY,CAD/HKD,-4,0.1
66,GBP_CAD,CURRENCY,GBP/CAD,-4,0.03


In [6]:
currencies = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']

In [7]:
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']

In [30]:
def fetch_candles(pair_name, granularity):
    global header
    url = f'{static_api}/instruments/{pair_name}/candles'
    params = {
        # 'count' : count,
        'granularity' : granularity,
        'price' : 'MBA',
        
    }
    response = session.get(url, params=params, headers=header)
    if response.status_code == 200:
        return response.json()
    else:
        print(response.status_code)
        return f'Error Code: {response.status_code}'

In [31]:
def get_candles_df(json_response):
    simple_candles = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for p in prices:
            for e in ohlc:
                new_dict[f'{p}_{e}'] = candle[p][e]
        simple_candles.append(new_dict)
    return pd.DataFrame.from_dict(simple_candles)

In [32]:
def save_candles(candles_df, pair, granularity):
    candles_df.to_pickle(f'candles_data/{pair}_{granularity}.pkl')

In [41]:
for p1 in currencies:
    for p2 in currencies:
        pair = f'{p1}_{p2}'
        if pair in ins_df.name.unique():
            create_data(pair, 'H1')

In [42]:
EUR_USD = pd.read_pickle('candles_data/EUR_USD_H1.pkl')

In [43]:
EUR_USD

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-05-02T00:00:00.000000000Z,9726,1.05349,1.05532,1.05267,1.05294,1.05339,1.05524,1.05258,1.05285,1.05359,1.05542,1.05276,1.05303
1,2022-05-02T01:00:00.000000000Z,7057,1.05294,1.05336,1.05216,1.05245,1.05284,1.05329,1.05208,1.05236,1.05304,1.05343,1.05225,1.05254
2,2022-05-02T02:00:00.000000000Z,4465,1.05244,1.05277,1.05180,1.05210,1.05236,1.05268,1.05169,1.05202,1.05253,1.05287,1.05190,1.05217
3,2022-05-02T03:00:00.000000000Z,5004,1.05210,1.05276,1.05174,1.05242,1.05202,1.05267,1.05165,1.05234,1.05219,1.05284,1.05182,1.05251
4,2022-05-02T04:00:00.000000000Z,4011,1.05242,1.05246,1.05169,1.05186,1.05233,1.05237,1.05159,1.05178,1.05251,1.05256,1.05178,1.05195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,2022-05-30T16:00:00.000000000Z,2736,1.07796,1.07866,1.07781,1.07864,1.07788,1.07857,1.07773,1.07855,1.07804,1.07874,1.07789,1.07872
495,2022-05-30T17:00:00.000000000Z,1211,1.07862,1.07870,1.07824,1.07842,1.07853,1.07861,1.07814,1.07833,1.07871,1.07878,1.07832,1.07851
496,2022-05-30T18:00:00.000000000Z,1087,1.07840,1.07848,1.07806,1.07834,1.07831,1.07839,1.07796,1.07826,1.07849,1.07857,1.07815,1.07843
497,2022-05-30T19:00:00.000000000Z,1675,1.07834,1.07838,1.07772,1.07792,1.07826,1.07830,1.07763,1.07782,1.07841,1.07847,1.07782,1.07803


In [44]:
EUR_USD['mid_o'][0]

'1.05349'

In [45]:
len(EUR_USD.index)

499

In [46]:
test = EUR_USD.copy()

In [47]:
non_cols = ['time', 'volume']
cols = [x for x in test.columns if x not in non_cols]
test[cols] = test[cols].apply(pd.to_numeric)

In [48]:
test.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-05-02T00:00:00.000000000Z,9726,1.05349,1.05532,1.05267,1.05294,1.05339,1.05524,1.05258,1.05285,1.05359,1.05542,1.05276,1.05303
1,2022-05-02T01:00:00.000000000Z,7057,1.05294,1.05336,1.05216,1.05245,1.05284,1.05329,1.05208,1.05236,1.05304,1.05343,1.05225,1.05254
2,2022-05-02T02:00:00.000000000Z,4465,1.05244,1.05277,1.05180,1.05210,1.05236,1.05268,1.05169,1.05202,1.05253,1.05287,1.05190,1.05217
3,2022-05-02T03:00:00.000000000Z,5004,1.05210,1.05276,1.05174,1.05242,1.05202,1.05267,1.05165,1.05234,1.05219,1.05284,1.05182,1.05251
4,2022-05-02T04:00:00.000000000Z,4011,1.05242,1.05246,1.05169,1.05186,1.05233,1.05237,1.05159,1.05178,1.05251,1.05256,1.05178,1.05195


In [49]:
def average_true_range(candles, length):
    if type(length) != int or length < 1:
        print('invalid length')
        return None

    tr = [abs(candles['mid_h'][0] - candles['mid_l'][0])]
    for i in range(1, len(candles.index)):
        a = abs(candles['mid_h'][i] - candles['mid_l'][i])
        b = abs(candles['mid_h'][i] - candles['mid_l'][i-1])
        c = abs(candles['mid_h'][i-1] - candles['mid_l'][i])
        tr.append(max(a,b,c))
    atr = [stat.mean(tr[x-length+1 : x+2]) if x >= length-1 else None for x in range(len(candles.index))]
    candles[f'atr_{length}'] = atr

    return

average_true_range(test, 14)
test.head(30)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,atr_14
0,2022-05-02T00:00:00.000000000Z,9726,1.05349,1.05532,1.05267,1.05294,1.05339,1.05524,1.05258,1.05285,1.05359,1.05542,1.05276,1.05303,NaN
1,2022-05-02T01:00:00.000000000Z,7057,1.05294,1.05336,1.05216,1.05245,1.05284,1.05329,1.05208,1.05236,1.05304,1.05343,1.05225,1.05254,NaN
2,2022-05-02T02:00:00.000000000Z,4465,1.05244,1.05277,1.05180,1.05210,1.05236,1.05268,1.05169,1.05202,1.05253,1.05287,1.05190,1.05217,NaN
3,2022-05-02T03:00:00.000000000Z,5004,1.05210,1.05276,1.05174,1.05242,1.05202,1.05267,1.05165,1.05234,1.05219,1.05284,1.05182,1.05251,NaN
4,2022-05-02T04:00:00.000000000Z,4011,1.05242,1.05246,1.05169,1.05186,1.05233,1.05237,1.05159,1.05178,1.05251,1.05256,1.05178,1.05195,NaN
5,2022-05-02T05:00:00.000000000Z,4486,1.05185,1.05214,1.05105,1.05134,1.05177,1.05206,1.05096,1.05123,1.05193,1.05222,1.05114,1.05144,NaN
6,2022-05-02T06:00:00.000000000Z,8676,1.05135,1.05518,1.05102,1.05486,1.05124,1.05509,1.05092,1.05477,1.05146,1.05529,1.05110,1.05494,NaN
7,2022-05-02T07:00:00.000000000Z,12803,1.05485,1.05685,1.05319,1.05324,1.05476,1.05675,1.05310,1.05314,1.05494,1.05696,1.05328,1.05334,NaN
8,2022-05-02T08:00:00.000000000Z,10719,1.05325,1.05362,1.05256,1.05290,1.05316,1.05355,1.05247,1.05280,1.05334,1.05372,1.05265,1.05300,NaN
9,2022-05-02T09:00:00.000000000Z,6264,1.05290,1.05330,1.05188,1.05270,1.05281,1.05322,1.05179,1.05262,1.05300,1.05339,1.05197,1.05278,NaN


In [51]:
def relative_strength_index(candles, length=14, price='mid'):
    change = [(candles[f'{price}_c'][row] - candles[f'{price}_o'][row]) for row in range(len(candles))]
    pos = [max(x, 0) for x in change]
    neg = [abs(min(x, 0)) for x in change]
    cag = stat.mean(pos[0: length])
    cal = stat.mean(neg[0: length])
    rs = [None] * (length - 1)
    rs.append(100 - (100 / (1 + (cag / cal))))

    for x in range((length), len(change)):
        cag = ((cag * 13) + pos[x]) / length
        cal = ((cal * 13) + neg[x]) / length

        rs.append(100 - (100 / (1 + (cag / (cal)))))

    candles[f'rsi_{length}'] = rs


relative_strength_index(test)
test.head(30)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,atr_14,rsi_14
0,2022-05-02T00:00:00.000000000Z,9726,1.05349,1.05532,1.05267,1.05294,1.05339,1.05524,1.05258,1.05285,1.05359,1.05542,1.05276,1.05303,NaN,NaN
1,2022-05-02T01:00:00.000000000Z,7057,1.05294,1.05336,1.05216,1.05245,1.05284,1.05329,1.05208,1.05236,1.05304,1.05343,1.05225,1.05254,NaN,NaN
2,2022-05-02T02:00:00.000000000Z,4465,1.05244,1.05277,1.05180,1.05210,1.05236,1.05268,1.05169,1.05202,1.05253,1.05287,1.05190,1.05217,NaN,NaN
3,2022-05-02T03:00:00.000000000Z,5004,1.05210,1.05276,1.05174,1.05242,1.05202,1.05267,1.05165,1.05234,1.05219,1.05284,1.05182,1.05251,NaN,NaN
4,2022-05-02T04:00:00.000000000Z,4011,1.05242,1.05246,1.05169,1.05186,1.05233,1.05237,1.05159,1.05178,1.05251,1.05256,1.05178,1.05195,NaN,NaN
5,2022-05-02T05:00:00.000000000Z,4486,1.05185,1.05214,1.05105,1.05134,1.05177,1.05206,1.05096,1.05123,1.05193,1.05222,1.05114,1.05144,NaN,NaN
6,2022-05-02T06:00:00.000000000Z,8676,1.05135,1.05518,1.05102,1.05486,1.05124,1.05509,1.05092,1.05477,1.05146,1.05529,1.05110,1.05494,NaN,NaN
7,2022-05-02T07:00:00.000000000Z,12803,1.05485,1.05685,1.05319,1.05324,1.05476,1.05675,1.05310,1.05314,1.05494,1.05696,1.05328,1.05334,NaN,NaN
8,2022-05-02T08:00:00.000000000Z,10719,1.05325,1.05362,1.05256,1.05290,1.05316,1.05355,1.05247,1.05280,1.05334,1.05372,1.05265,1.05300,NaN,NaN
9,2022-05-02T09:00:00.000000000Z,6264,1.05290,1.05330,1.05188,1.05270,1.05281,1.05322,1.05179,1.05262,1.05300,1.05339,1.05197,1.05278,NaN,NaN
